In [1]:
%matplotlib inline
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal, stats
from scipy.signal import butter, lfilter, freqz
from fractions import Fraction
from numpy.lib import stride_tricks
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import RFECV, mutual_info_classif, SelectKBest
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC
from sklearn import tree
import plotly.offline as py
py.init_notebook_mode()
import plotly.graph_objs as go

### <center>Dirty jobs: the reality of working with wearable activity tracker data</center>
#### <center>MD Sherman</center>

In [2]:
sig = pd.read_csv("./rawDataForMarcus.csv", index_col=False).transpose()
sig = sig.reset_index(np.arange(0,len(sig)))
sig.columns = np.arange(0, sig.shape[1])
labels = sig[7000]
sig = sig.iloc[:,900:6660]
pos = sig[labels == 1]
neg = sig[labels == 0]
neg = neg.sample(n = pos.shape[0])
sig = pd.concat([pos, neg])
labels = labels.iloc[sig.index].sort_index().reset_index(drop = True)
Fs = Fraction(1, 60)
sig = pd.DataFrame(sig.iloc[:,:7000].sort_index(), dtype = 'float64').reset_index(drop = True)

In [6]:
#cutoff = .2
#Order = 3
#B, A = signal.butter(Order, cutoff, output='ba')
#sigf = pd.DataFrame(signal.filtfilt(B,A, sig))
#sigf.columns = sig.columns

### Cleaning up the signal

In [4]:
traces = []
traces.append(go.Scatter(y = sig.iloc[1,:], opacity = 0.5, name = 'Raw'))
for i in range(1,7):
    B, A = signal.butter(i, cutoff, output='ba')
    sigd = sigf = signal.filtfilt(B,A, sig)
    traces.append(go.Scatter(y = sigd[1,:], opacity = 0.5, name = 'Order: %d' % (i)))
layout = go.Layout(
    width = 1000,
    height = 700,
    title = 'Raw vs Filtered (0-20 Hz) armband data',
    xaxis = dict(
        title = 'Time (min)'
    ),
    yaxis = dict(
        title = 'Sum of 3-axis Acceleration (m/s<sup>2</sup>)'
    ),
    showlegend = False
)
figure = go.Figure(data = traces, layout = layout)
py.iplot(figure)

In [3]:
def welch(series):
    f, Pxx_spec = signal.welch(series.values, Fs, 'flattop', 128, scaling = 'spectrum', return_onesided = True)
    return np.sqrt(np.max(Pxx_spec)) - np.sqrt(np.min(Pxx_spec))

    
def autocorr(series):
    return series.autocorr(lag = 5)


def delta(series):
    return max(series) - min(series)


def power(series):
    return sum(np.multiply(series, np.conj(series)))


def rms(series):
    return np.sqrt(np.mean(series**2))


def rms_freq(series):
    n = len(series)
    Y = abs(np.fft.rfft(series)/n)
    return np.sqrt(np.mean(Y**2))


def meanFreq(series):
    n = len(series)
    Y = abs(np.fft.rfft(series)/n)
    return np.mean(Y)


def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    
    # zeros at beginning (thus center of 1st window should be for sample nr. 0)
    samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = int(np.ceil( (len(samples) - frameSize) / float(hopSize))) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))
    
    frames = stride_tricks.as_strided(samples, shape=(cols, frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win
    
    return np.fft.rfft(frames) 


def spectral_flux(wavedata, sample_rate):
    wavedata = wavedata.values
    window_size = len(wavedata)
    
    # convert to frequency domain
    magnitude_spectrum = stft(wavedata, window_size)
    timebins, freqbins = np.shape(magnitude_spectrum)
    
    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,timebins - 1) * (timebins / float(sample_rate)))
    
    sf = np.sqrt(np.sum(np.diff(np.abs(magnitude_spectrum))**2, axis=1)) / freqbins
    
    return sf[1:]


def spectral_rolloff(wavedata, sample_rate, k=0.85):
    wavedata = wavedata.values
    window_size = len(wavedata)
    
    # convert to frequency domain
    magnitude_spectrum = stft(wavedata, window_size)
    power_spectrum   = np.abs(magnitude_spectrum)**2
    timebins, freqbins = np.shape(magnitude_spectrum)
    
    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,timebins - 1) * (timebins / float(sample_rate)))
    
    sr = []

    spectralSum = np.sum(power_spectrum, axis=1)
    
    for t in range(timebins-1):
        
        # find frequency-bin indeces where the cummulative sum of all bins is higher
        # than k-percent of the sum of all bins. Lowest index = Rolloff
        sr_t = np.where(np.cumsum(power_spectrum[t,:]) >= k * spectralSum[t])[0][0]
        
        sr.append(sr_t)
        
    sr = np.asarray(sr).astype(float)
    
    # convert frequency-bin index to frequency in Hz
    sr = (sr / freqbins) * (sample_rate / 2.0)
    
    return sr


def spectral_centroid(wavedata, sample_rate):
    wavedata = wavedata.values
    window_size = len(wavedata)
    
    magnitude_spectrum = stft(wavedata, window_size)
    
    timebins, freqbins = np.shape(magnitude_spectrum)
    
    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,timebins - 1) * (timebins / float(sample_rate)))
    
    sc = []

    for t in range(timebins-1):
        
        power_spectrum = np.abs(magnitude_spectrum[t])**2
        
        sc_t = np.sum(power_spectrum * np.arange(1,freqbins+1)) / np.sum(power_spectrum)
        
        sc.append(sc_t)
        
    sc = np.asarray(sc)
    sc = np.nan_to_num(sc)
    
    return sc


def feature_gen(dataFrame, name=''):
    dataFrame = pd.DataFrame(dataFrame)
    df = pd.DataFrame(dtype='float64')
    metrics = stats.describe(dataFrame, axis = 1)
    df['%smean' %(name)] = [mu for mu in metrics.mean]
    df['%smedian' %(name)] = dataFrame.median(axis = 1)
    df['%svar' %(name)] = [var for var in metrics.variance]
    df['%sskewness' %(name)] = [mu for mu in metrics.skewness]
    df['%skurtosis' %(name)] = [mu for mu in metrics.kurtosis]
    df['%smax' %(name)] = [maxV for maxV in metrics.minmax[1]]
    df['%sdelta'%(name)] = dataFrame.apply(delta, axis = 1)
    df['%sauto'%(name)] = dataFrame.apply(autocorr, axis = 1)
    df['%spow_total'%(name)] = dataFrame.apply(power, axis = 1)
    df['%srms'%(name)] = dataFrame.apply(rms, axis = 1)
    df['%smean_freq'%(name)] = dataFrame.apply(meanFreq, axis = 1)
    df['%sPSD_delta'%(name)] = dataFrame.apply(welch, axis = 1)
    df['%sspec_flux'%(name)]  = dataFrame.apply(spectral_flux, axis = 1, args = (Fs,)).mean(axis = 1)
    df['%sspec_rolloff'%(name)] = dataFrame.apply(spectral_rolloff, axis = 1, args = (Fs,)).mean(axis = 1)
    df['%sspec_centroid'%(name)] = dataFrame.apply(spectral_centroid, axis = 1, args = (Fs,)).mean(axis = 1)
    df['%srms_freq'%(name)] = dataFrame.apply(rms_freq, axis  = 1)
    return df


def features(dataFrame):
    feats = pd.DataFrame(dtype='float64')
    #feats = []
    #feats = pd.DataFrame(index = dataFrame.index)
    feats = feature_gen(dataFrame)
    days = ['day1_', 'day2_', 'day3_', 'day4_']
    j = 0
    for i in range(0, dataFrame.shape[1], 1440):
        feats = pd.concat([feats, feature_gen(dataFrame.iloc[:, i:i + 1440], name=days[j])], axis = 1)
        j += 1
    times = {'a1_': (350, 1250), 's1_':(1250, 1750), 'a2_': (1750, 2650), 's2_': (2650, 3150), 'a3_': (3150, 4050), 's3_': (4050, 4550), 'a4_': (4550, 5400)}
    for k, v in times.items():
        feats = pd.concat([feats, feature_gen(dataFrame.iloc[:, v[0]:v[1]], name=k)], axis = 1)
       
    features = ['mean', 'median', 'var', 'skewness', 'kurtosis', 'max', 'delta', 'auto', 'pow_total',
                'rms', 'mean_freq', 'PSD_delta', 'spec_flux', 'spec_rolloff', 'spec_centroid', 'rms_freq']
    
    d = 0
    for day in days[0:3]:
        for feature in features:
            feats['%s-%s_%s' %(day, days[d+1], feature)] = feats['%s%s' %(day, feature)] - feats['%s%s' % (days[d+1], feature)]
        d += 1
    
    t = 0
    for time in list(times.keys())[0:6]:
        for feature in features:
            #print(time, list(times.keys())[t+1], feature, '\n', feats['%s%s' %(time, feature)] - feats['%s%s' % (list(times.keys())[d+1], feature)])
            feats['%s-%s_%s' %(time, list(times.keys())[t+1], feature)] = feats['%s%s' %(time, feature)] - feats['%s%s' % (list(times.keys())[t+1], feature)]
        t += 1
    
    return feats

In [4]:
sig_feat = features(sig)

### Feature selection with Mutual Information

In [24]:
ys= mutual_info_classif(sig_feat, labels)#[feat_space]
#ys = ys[ys > 0.15]
mYs = [round(np.mean(ys),2)]*336
layout = go.Layout(
    width = 1000,
    height = 700,
    autosize=False,
    title = 'Feature Mutual information (MI)',
    xaxis = dict(
        title = 'Feature'
    ),
    yaxis = dict(
        title = 'MI'
    )
)
fig = go.Figure(data = [go.Scatter( y = ys, mode = 'markers')], layout = layout)
py.iplot(fig)

#pd.Series(mutual_info_classif(sig_feat, labels)).plot()

In [189]:
fail = sig[labels == 0].median()
success = sig[labels == 1].median()

### Synchronizing and Exploring the data

In [190]:
trace3 = go.Scatter(
    x = np.arange(0, len(fail)),
    y = fail,
    name = 'Failures'
)
trace4 = go.Scatter(
    x = np.arange(0, len(success)),
    y = success,
    opacity = 0.5,
    name = 'Successes'
)
Layout = go.Layout(
    height = 700,
    width = 1000,
    title = 'Synchronized median of successes vs failures',
    xaxis = dict(
        title = 'Time (min)'
    ),
    yaxis = dict(
        title = 'Sum of 3-axis Acceleration (m/s<sup>2</sup>)'
    )
)
fig = go.Figure(data = [trace3, trace4], layout=Layout)
py.iplot(fig)

## Training & Optimization

In [162]:
# 42 -> Test = 12
sss1 = StratifiedShuffleSplit(n_splits=2, test_size=12)
for train, test in sss1.split(sig_feat, labels):
    x_downstream, x_test = sig_feat.iloc[train,:], sig_feat.iloc[test,:]
    y_downstream, y_test = labels.iloc[train], labels.iloc[test] 
# 30 -> Feature Selection = 6
sss2 = StratifiedShuffleSplit(n_splits=2, test_size=6)
for train, test in sss2.split(x_downstream, y_downstream):
    x_model, x_fs = x_downstream.iloc[train,:], x_downstream.iloc[test,:]
    y_model, y_fs = y_downstream.iloc[train], y_downstream.iloc[test] 
# 24 -> Optimize = 10, Train = 14    
sss3 = StratifiedShuffleSplit(n_splits=2, test_size=10)
for train, test in sss3.split(x_model, y_model):
    x_train, x_optimize = x_model.iloc[train,:], x_model.iloc[test,:]
    y_train, y_optimize = y_model.iloc[train], y_model.iloc[test] 


### KBest mutual information feature selection

In [163]:
# use feature selection partition
feat_space = SelectKBest(mutual_info_classif, k=21).fit(x_fs, y_fs).get_support()

### GBT Optimization, Training, and Testing

In [165]:
print("Optimizing")
# use model optimization partition
param_gbt = {'n_estimators' : [5, 50, 100, 150, 200, 250, 300],
            'learning_rate': np.arange(0.01, 0.21, 0.01),
            #'max_depth': range(3,8)
            }
gsearch_gbt = GridSearchCV(
    estimator=GradientBoostingClassifier(), 
    param_grid=param_gbt, scoring='roc_auc', n_jobs=-1, cv = 6, refit=True)
gsearch_gbt.fit(x_model.iloc[:,feat_space].values, y_model.values)

print('Training')
# use training partition
#clf_gbt = GradientBoostingClassifier(**gsearch_gbt.best_params_)
#clf_gbt.fit(x_train.iloc[:,feat_space], y_train)

print('Testing')
# now use testing partition
print(gsearch_gbt.best_params_)
y_score = gsearch_gbt.predict_proba(x_test.iloc[:,feat_space])[:,1]
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_KB_gbt = go.Scatter(x = fpr, y = tpr, name = 'GBT')

Optimizing
Training
Testing
{'learning_rate': 0.17000000000000001, 'n_estimators': 5}
AUC = 0.458333


In [166]:
y_score = gsearch_gbt.decision_function(x_test.iloc[:,feat_space].values)
fpr, tpr, _ = roc_curve(y_test.values, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_KB_gbt = go.Scatter(x = fpr, y = tpr, name = 'GBT')

AUC = 0.458333


### Recursive feature elimination with Cross-validation

In [167]:
# use the downstream partition since feature selection and model training are done at the same time
tuned = GradientBoostingClassifier(**gsearch_gbt.best_params_)
rfecv = RFECV(estimator=tuned, cv = 6, scoring='roc_auc', n_jobs=-1)
rfecv.fit(x_model.iloc[:,feat_space], y_model.values)

# now see how well RFECV did with test partition
y_score = rfecv.decision_function(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))

layout = go.Layout(
    width = 1000,
    height = 700,
    title = 'RFECV feature importance and ranking',
    xaxis = dict(
        title = 'Number of Features'
    ),
    yaxis = dict(
        title = "Cross validation score (# correct classifications)"
    )
)
fig = go.Figure(data = [go.Scatter(x = list(range(1, len(rfecv.grid_scores_) + 1)), y = rfecv.grid_scores_)], layout = layout)
py.iplot(fig)
trace_rfecv_gbt = go.Scatter(x = fpr, y = tpr, name = 'RFECV')

AUC = 0.486111


In [47]:
y_score = rfecv.decision_function(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))

AUC = 0.472222


### Random Forest

In [168]:
# use model optimization partition
params_rf= {'max_features': range(5,21), 'max_depth': range(1,5), 
            'min_samples_split': range(2,5), 'min_samples_leaf': range(1,6),
            'class_weight': [None, 'balanced']}
gsearch_rf = GridSearchCV(
    estimator=RandomForestClassifier(criterion='entropy'), 
    param_grid=params_rf, scoring='roc_auc', n_jobs=-1, cv = 6, refit=True)
gsearch_rf.fit(x_model.iloc[:,feat_space], y_model)

y_score = gsearch_rf.predict_proba(x_test.iloc[:,feat_space])[:,0]
fpr, tpr, _ = roc_curve(y_test.values, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_rf = go.Scatter(x = fpr, y = tpr, name = 'rf')

AUC = 0.666667


### Logistic regression

In [169]:
print("Optimizing")
# Use optimize partition
params_log= {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 1000],
             'max_iter': [5, 50, 100, 250, 500, 750, 1000]}
gsearch_log = GridSearchCV(
    estimator=LogisticRegression(class_weight='balanced', solver='liblinear'), 
    param_grid=params_log, scoring='roc_auc', n_jobs=4, cv = 6, refit=True)
gsearch_log.fit(x_model.iloc[:,feat_space], y_model)

print('Training')
# use train partition
#clf_log = LogisticRegression(**gsearch_log.best_params_, class_weight='balanced')
#clf_log.fit(x_train.iloc[:,feat_space], y_train)

print('Testing')
# use test partition
y_score = gsearch_log.decision_function(x_test.iloc[:,feat_space].values)
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_log = go.Scatter(x = fpr, y = tpr, name = 'log')

Optimizing
Training
Testing
AUC = 0.555556


### SVC

In [181]:
from sklearn import preprocessing
print("Optimizing")
# use optimize partition
params_svc= {'C': np.logspace(-2, 10, 13),
            'gamma': np.logspace(-9, 3, 13),
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
            'degree': range(3,13), 'max_iter': [-1, 10, 100, 1000, 10000],
            }
gsearch_svc = GridSearchCV(
    estimator=SVC(), 
    param_grid=params_svc, scoring='accuracy', n_jobs=-1, cv = 6, refit=True)
svc_x = preprocessing.MinMaxScaler().fit_transform(x_model.iloc[:,feat_space])
gsearch_svc.fit(svc_x, y_model)

print('Training')
# use train partition
#clf_svc = SVC(**gsearch_svc.best_params_)
#clf_svc.fit(x_train.iloc[:,feat_space], y_train)

print('Testing')
# use test partition
y_score = gsearch_svc.decision_function(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_scv = go.Scatter(x = fpr, y = tpr, name = 'SVC')

Optimizing
Training
Testing
AUC = 0.500000


### CART

In [170]:
print("Optimizing")
params_tree= {'max_features': range(5,21), 'max_depth': range(1,16), 
              'min_samples_split': range(2,6), 'min_samples_leaf': range(1,6),
             'splitter': ['best', 'random']}
gsearch_tree = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(criterion='entropy', class_weight='balanced'), 
    param_grid=params_tree, scoring='roc_auc', n_jobs=-1, cv = 6, refit=True)
gsearch_tree.fit(x_model.iloc[:,feat_space], y_model)

print('Training')
# use train partition
#clf_tree = tree.DecisionTreeClassifier(**gsearch_tree.best_params_)
#clf_tree.fit(x_train.iloc[:,feat_space], y_train)

print('Testing')
# use test partition
y_score = gsearch_tree.predict_proba(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
print("AUC = %f" %(auc(fpr, tpr)))
trace_cart = go.Scatter(x = fpr, y = tpr, name = 'CART')

Optimizing
Training
Testing
AUC = 0.541667


### Naive KNN

In [171]:
print("Optimizing")
# use optimize partition
params_knn= {'n_neighbors': range(2, 15), 'leaf_size': range(1,35,5),
            'p': range(1,6)}
gsearch_knn = GridSearchCV(
    estimator=KNeighborsClassifier(weights='distance', algorithm='ball_tree'), 
    param_grid=params_knn, n_jobs=-1, cv = 3, refit=True)
gsearch_knn.fit(x_model.iloc[:,feat_space], y_model)

print('Training')
# use train partition
#clf_knn = KNeighborsClassifier(**gsearch_knn.best_params_)
#clf_knn.fit(x_train.iloc[:,feat_space], y_train)

print('Testing')
# use test partition
y_score = gsearch_knn.predict_proba(x_test.iloc[:,feat_space])[:,0]
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_knn = go.Scatter(x = fpr, y = tpr, name = 'KNN')

Optimizing
Training
Testing
AUC = 0.583333


In [175]:
from sklearn.naive_bayes import GaussianNB

gauss = GaussianNB().fit(x_model.iloc[:,feat_space], y_model)

print(cross_val_score(GaussianNB(),x_model.iloc[:,feat_space], y_model))

#gsearch_nbayes.fit(x_model.iloc[:,feat_space], y_model)

y_score = gauss.predict_proba(x_test.iloc[:,feat_space])[:,1]
fpr, tpr, _ = roc_curve(y_test, y_score)
print("AUC = %f" %(auc(fpr, tpr)))
trace_nbayes = go.Scatter(x = fpr, y = tpr, name = 'Naive Bayes')

[ 0.5    0.625  0.375]
AUC = 0.638889


### Multi-layer perceptron

In [176]:
print("Optimizing")
params_mlp = {'hidden_layer_sizes': range(100, 150, 5), 'activation': ['relu', 'logistic'],
              'solver': ['sgd', 'adam'], 'alpha' : [0.0001, 0.00015, 0.001],
              'learning_rate': ['constant', 'invscaling', 'adaptive'],
              'max_iter': range(100, 300, 25)}
gsearch_mlp = GridSearchCV(
    estimator=MLPClassifier(early_stopping=True), 
    param_grid=params_mlp, scoring='roc_auc', n_jobs=4, cv = 5, refit=True)
gsearch_mlp.fit(x_model.iloc[:,feat_space], y_model)

print('Training')
# use train partition
#clf_mlp = MLPClassifier(early_stopping=True, **gsearch_mlp.best_params_)
#clf_mlp.fit(x_train.iloc[:,feat_space], y_train)

print('Testing')
# use test partition
y_score = gsearch_mlp.predict_proba(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
print("AUC = %f" %(auc(fpr, tpr)))
trace_mlp = go.Scatter(x = fpr, y = tpr, name = 'mlp')

Optimizing
Training
Testing
AUC = 0.472222


In [ ]:
y_score = eclf.predict_proba(x_test.iloc[:,feat_space])[:,1]
fpr, tpr, _ = roc_curve(y_test)
print("AUC = %f" %(auc(fpr, tpr)))
py.iplot([go.Scatter(x = fpr, y = tpr, name = 'mlp')])

In [178]:
from sklearn.ensemble import VotingClassifier
cart = gsearch_tree.best_estimator_
log = gsearch_log.best_estimator_
gbt = gsearch_gbt.best_estimator_
mlp = gsearch_mlp.best_estimator_
knn = gsearch_knn.best_estimator_
svc = gsearch_svc.best_estimator_
rf = gsearch_rf.best_estimator_
voter = VotingClassifier(estimators=[
         ('cart', cart), ('log', log), ('gbt', gbt), ('mlp', mlp),
('knn', knn), ('rf', rf)], voting='soft', n_jobs = -1)
voter.fit(x_model.iloc[:,feat_space], y_model)

#voter.score(x_test.iloc[:,feat_space], y_test)
y_score = voter.predict_proba(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
print("AUC = %f" %(auc(fpr, tpr)))
trace_voter = go.Scatter(x = fpr, y = tpr, name = 'Voter')

AUC = 0.500000


In [182]:
from sklearn.ensemble import BaggingClassifier
params_bag = {'n_estimators': range(5,55, 5), 'base_estimator': [cart, log, gbt, mlp, knn, rf]}
gsearch_bag = GridSearchCV(
    estimator=BaggingClassifier(), 
    param_grid=params_bag, scoring='roc_auc', n_jobs=-1, cv = 6)
gsearch_bag.fit(x_model.iloc[:,feat_space], y_model)

y_score = gsearch_bag.predict_proba(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
print("AUC = %f" %(auc(fpr, tpr)))
trace_bag = go.Scatter(x = fpr, y = tpr, name = 'bag')

AUC = 0.555556


In [183]:
from mlxtend.classifier import StackingClassifier
cart = gsearch_tree.best_estimator_
log = gsearch_log.best_estimator_
gbt = gsearch_gbt.best_estimator_
mlp = gsearch_mlp.best_estimator_
knn = gsearch_knn.best_estimator_
svc = gsearch_svc.best_estimator_
rf = gsearch_rf.best_estimator_
sclf = StackingClassifier(classifiers=[cart, log, gbt, mlp, knn, rf],
                          meta_classifier=KNeighborsClassifier())
sclf.fit(x_model.iloc[:,feat_space], y_model)

y_score = sclf.predict_proba(x_test.iloc[:,feat_space])
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
print("AUC = %f" %(auc(fpr, tpr)))
trace_stack = go.Scatter(x = fpr, y = tpr, name = 'stack')

AUC = 0.500000


In [160]:
for label, clf in [('cart', cart), ('log', log), ('gbt', gbt),
                   ('mlp', mlp),('knn', knn), ('rf', rf), ('stacker', sclf)]:
    scores = cross_val_score(clf, x_model.iloc[:,feat_space], y_model, cv=5, scoring='roc_auc')
    print("AUCROC: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

AUCROC: 0.54 (+/- 0.25) [cart]
AUCROC: 0.41 (+/- 0.24) [log]
AUCROC: 0.75 (+/- 0.31) [gbt]
AUCROC: 0.50 (+/- 0.00) [mlp]
AUCROC: 0.53 (+/- 0.19) [knn]
AUCROC: 0.76 (+/- 0.26) [rf]
AUCROC: 0.74 (+/- 0.32) [stacker]


In [188]:
traces = [trace_KB_gbt, trace_rfecv_gbt, trace_log, 
          trace_scv, trace_cart, trace_rf, trace_knn, trace_mlp,
         trace_voter, trace_bag, trace_stack]
Layout = go.Layout(
    height = 700,
    width = 1000,
    title = 'ROC plots of selected classifiers',
    xaxis = dict(
        title = 'False Positive Rate (FPR)'),
    yaxis = dict(
        title = 'True Positive Rate (TPR)'),
    shapes = [{
        'type': 'line',
        'x0': 0,
        'x1': 1,
        'y0': 0,
        'y1': 1,
        'line': {
            'width': 4,
            'dash': 'dash'}
    }]
)
fig = go.Figure(data = traces, layout = Layout)
py.iplot(fig)